In [1]:
import torch
import numpy as np
from abc import ABC, abstractmethod

In [2]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [3]:
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass

In [4]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [5]:
def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)

In [6]:
def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out)

In [7]:
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        
        converted_kernel = 0
        
        for filtr in range(self.kernel.shape[0]):
            for channel in range(self.kernel.shape[1]):
                for i in range(self.kernel.shape[2]):
                    if not torch.is_tensor(converted_kernel):
                        converted_kernel = self.kernel[filtr][channel][i]
                    else:
                        converted_kernel = torch.cat((converted_kernel, self.kernel[filtr][channel][i]), 0)
                        
        return converted_kernel.reshape((1, -1))

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        
        converted_input = 0
        outputs_number = int(1 + int((torch_input.shape[3] - self.kernel.shape[3])/self.stride))**2
        
        
        for image in range(torch_input.shape[0]):
            converted_image = 0
            for channel in range(torch_input.shape[1]):
                converted_input_auxiliary = 0
                for i in range(0, int(np.sqrt(outputs_number)), self.stride):
                    for j in range(0, int(np.sqrt(outputs_number)), self.stride):
                        
                        if not torch.is_tensor(converted_input_auxiliary):
                            converted_input_auxiliary = torch_input[image][channel][i:i+self.kernel_size, i:i+self.kernel_size].reshape(-1, 1)
                        else:
                            tmp = torch_input[image][channel][i:i+self.kernel_size, j:j+self.kernel_size].reshape(-1, 1)
                            converted_input_auxiliary = torch.cat((converted_input_auxiliary, tmp), 1)
                        
                        if (i + self.stride > int(np.sqrt(outputs_number))) and (j + self.stride > int(np.sqrt(outputs_number))):
                            if not torch.is_tensor(converted_image):
                                converted_image = converted_input_auxiliary
                            else:
                                converted_image = torch.cat((converted_image, converted_input_auxiliary), 0)
                                
            if not torch.is_tensor(converted_input):
                converted_input = converted_image
            else:
                converted_input = torch.cat((converted_input, converted_image), 1)

        return converted_input


        
    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(0, 1).view(torch_input.shape[0],
                                                     self.out_channels, output_height,
                                                     output_width).transpose(1, 3).transpose(2, 3)

In [8]:
print(test_conv2d_layer(Conv2dMatrixV2))

True
